# SVM Emotion Recognition — Training Notebook

In [ ]:
# !pip install kagglehub pandas numpy imbalanced-learn
# For COLAB USE

In [ ]:
import kagglehub

csv_path = kagglehub.dataset_download("davilsena/ckdataset")

print("Path to dataset files:", csv_path)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib

try:
    from imblearn.over_sampling import SMOTE
    has_smote = True
except:
    has_smote = False

TARGET_SIZE = (48, 48)


import os
for dirname, _, filenames in os.walk('/kaggle/input/ckdataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
CSV_PATH = "/kaggle/input/ckdataset/ckextended.csv"  # update your dataset path here
MODEL_PATH = '/content/'
SCALER_PATH = '/content/'

df = pd.read_csv(CSV_PATH)
pixels = df['pixels'].apply(lambda s: np.fromstring(s, sep=' ', dtype=int))
X = np.stack(pixels.values).astype('float32') / 255.0
y = df['emotion'].astype(int).values
print('Dataset loaded:', X.shape, y.shape)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

if has_smote:
    sm = SMOTE(random_state=42)
    X_train_scaled, y_train = sm.fit_resample(X_train_scaled, y_train)
    print('Applied SMOTE.')

Dataset loaded: (920, 2304) (920,)


In [ ]:
svm = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
svm.fit(X_train_scaled, y_train)

y_pred = svm.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

Applied SMOTE.


In [ ]:
import joblib
import os

drive_path = '/content/drive/My Drive/Colab_Saved_Models/'
os.makedirs(drive_path, exist_ok=True)

scaler_filename = 'scaler.joblib'
scaler_output_path = os.path.join(drive_path, scaler_filename)
joblib.dump(scaler, scaler_output_path)
print(f"Scaler saved successfully to: {scaler_output_path}")

model_filename = 'svm_emotion_detector_resampled.joblib'
output_path = os.path.join(drive_path, model_filename)
joblib.dump(svm, output_path)
print(f"Model saved successfully to: {output_path}")